In [1]:
import requests 
import pandas as pd
import re
import warnings
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options


warnings.simplefilter('ignore')
from bs4 import BeautifulSoup as bs

cols = ['price','make_model','year','type','mileage','engine_size','power','gearbox','fuel_type','number_of_owners','location','combined_mpg']
auto_trader_df = pd.DataFrame(columns=cols)

for postcode in postcodes:
    print(f"Postcode: {postcode}")
    for page in range(1,2):
        print(f"Page: {page}")
        url = f"https://www.autotrader.co.uk/car-search?postcode={postcode}&radius=50&include-delivery-option=on&page={page}"
        pages = requests.get(url,headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
        soup = bs(pages.content, "html.parser")

        price = []
        make_model = []
        year = []
        vehicle_type = []
        mileage = []
        engine_size = []
        power = []
        gearbox = []
        fuel_type = []
        num_of_owners = []
        location = []

        for item in soup.find_all("li", class_ = 'search-page__result'):
            if len(item.find_all("li", class_ = 'atc-type-picanto--medium')) == 8:
                try:
                    price.append(item.find("div", class_ = 'product-card-pricing__price').text.split('\n')[1])
                    make_model.append(item.find("h3", class_ = 'product-card-details__title').text.split('\n')[1])
                    specs = item.find_all("li", class_ = 'atc-type-picanto--medium')
                    location.append(item.find_all("li", class_ = 'product-card-seller-info__spec-item atc-type-picanto')[-1].text.split('\n')[4].title())

                    listing_url = item.find("a", class_ = 'js-click-handler listing-fpa-link tracking-standard-link').get('href')
                    listing_page = requests.get(f"https://www.autotrader.co.uk{listing_url}",headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
                    listing_soup = bs(listing_page.content, "html.parser")

                    driver.get(f"https://www.autotrader.co.uk{listing_url}")

                    WebDriverWait(driver,20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id='sp_message_iframe_687971']")))
                    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[aria-label='Accept All']"))).click()
                    
                    driver.execute_script("window.scrollTo(100,1200);")
                    time.sleep(5)

                    span_tag = driver.find_element(By.XPATH, "//*[contains(text(), 'Running costs')]")
                    span_tag.find_element(By.XPATH, "..").click()

                    time.sleep(5)
                    mpg = driver.find_elements(By.XPATH, "//*[contains(text(), 'mpg')]")[-1]

                    combined_mpg.append(mpg.text)

                except:
                    pass

                try:
                    year.append(specs[0].text)
                    vehicle_type.append(specs[1].text)
                    mileage.append(specs[2].text)
                    engine_size.append(specs[3].text)
                    power.append(specs[4].text)
                    gearbox.append(specs[5].text)
                    fuel_type.append(specs[6].text)
                    num_of_owners.append(specs[7].text)
                except:
                    pass
            else:
                pass


        for index in range(len(price)):
            try:
                auto_trader_df = auto_trader_df.append({'price':price[index],'make_model':make_model[index],'year':year[index],'type':vehicle_type[index],'mileage':mileage[index],'engine_size':engine_size[index],'power':power[index],'gearbox':gearbox[index],'fuel_type':fuel_type[index],'number_of_owners':num_of_owners[index],'location':location[index],'combined_mpg':combined_mpg[index]},ignore_index=True)
            except:
                pass

auto_trader_df['price'] = auto_trader_df.price.str.replace('£','')
auto_trader_df['price'] = auto_trader_df.price.str.replace(',','')
auto_trader_df['price'] = pd.to_numeric(auto_trader_df['price'])

auto_trader_df['year'] = auto_trader_df['year'].apply(lambda year: year[:4])
auto_trader_df['year'] = pd.to_numeric(auto_trader_df['year'])

auto_trader_df['mileage'] = auto_trader_df['mileage'].apply(lambda mileage: mileage[:-6])
auto_trader_df['mileage'] = auto_trader_df.mileage.str.replace(',','')
auto_trader_df['mileage'] = pd.to_numeric(auto_trader_df['mileage'])

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

# Engine size is shown in Litres
auto_trader_df['engine_size'] = auto_trader_df.engine_size.str.replace('L','')
auto_trader_df['engine_size'] = pd.to_numeric(auto_trader_df['engine_size'])

auto_trader_df['number_of_owners'] = auto_trader_df['number_of_owners'].apply(lambda num_of_owners: num_of_owners[0])
auto_trader_df['number_of_owners'] = pd.to_numeric(auto_trader_df['number_of_owners'], errors='coerce')

# Power will be converted to BHP if the value is in PS units
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: int(power[:-2])/1.014 if 'PS' in power else power)
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: power[:-3] if 'BHP' in str(power) else power)
auto_trader_df['power'] = pd.to_numeric(auto_trader_df['power'])

auto_trader_df['make'] = auto_trader_df['make_model'].apply(lambda make: make.split()[0])
auto_trader_df['model'] = auto_trader_df['make_model'].apply(lambda make: ' '.join(make.split()[1:]))
auto_trader_df.drop(columns='make_model',inplace=True)

auto_trader_df['combined_mpg'] = auto_trader_df['combined_mpg'].apply(lambda mpg: mpg.split()[0])
auto_trader_df['combined_mpg'] = pd.to_numeric(auto_trader_df['combined_mpg'])

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

for index,make in enumerate(auto_trader_df['make']):
    if re.match(r"Mazda[0-9]+",make):
        auto_trader_df['make'][index] = re.split('(\d+)',make)[0]
        auto_trader_df['model'][index] = re.split('(\d+)',make)[1]
    else:
        pass

warnings.simplefilter('ignore')
from bs4 import BeautifulSoup as bs

cols = ['price','make_model','year','type','mileage','engine_size','power','gearbox','fuel_type','number_of_owners','location','listing_href']
auto_trader_df = pd.DataFrame(columns=cols)

for postcode in postcodes:
    print(f"Postcode: {postcode}")
    for page in range(1,101):
        print(f"Page: {page}")
        url = f"https://www.autotrader.co.uk/car-search?postcode={postcode}&radius=50&include-delivery-option=on&page={page}"
        pages = requests.get(url,headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
        soup = bs(pages.content, "html.parser")

        price = []
        make_model = []
        year = []
        vehicle_type = []
        mileage = []
        engine_size = []
        power = []
        gearbox = []
        fuel_type = []
        num_of_owners = []
        location = []
        listing_href = []
        combined_mpg = []

        for item in soup.find_all("li", class_ = 'search-page__result'):
            if len(item.find_all("li", class_ = 'atc-type-picanto--medium')) == 8:
                try:
                    price.append(item.find("div", class_ = 'product-card-pricing__price').text.split('\n')[1])
                    make_model.append(item.find("h3", class_ = 'product-card-details__title').text.split('\n')[1])
                    specs = item.find_all("li", class_ = 'atc-type-picanto--medium')
                    location.append(item.find_all("li", class_ = 'product-card-seller-info__spec-item atc-type-picanto')[-1].text.split('\n')[4].title())

                    listing_url = item.find("a", class_ = 'js-click-handler listing-fpa-link tracking-standard-link').get('href')
                    listing_href.append(f"https://autotrader.co.uk{listing_url}")

                    year.append(specs[0].text)
                    vehicle_type.append(specs[1].text)
                    mileage.append(specs[2].text)
                    engine_size.append(specs[3].text)
                    power.append(specs[4].text)
                    gearbox.append(specs[5].text)
                    fuel_type.append(specs[6].text)
                    num_of_owners.append(specs[7].text)
                except:
                    pass
            else:
                pass


        for index in range(len(price)):
            try:
                auto_trader_df = auto_trader_df.append({'price':price[index],'make_model':make_model[index],'year':year[index],'type':vehicle_type[index],'mileage':mileage[index],'engine_size':engine_size[index],'power':power[index],'gearbox':gearbox[index],'fuel_type':fuel_type[index],'number_of_owners':num_of_owners[index],'location':location[index],'listing_href':listing_href[index]},ignore_index=True)
            except:
                pass
            
postcodes  = ["BT71 5LT",
"PH21 1LZ",
"ML11 0HN",
"CA9 3ND",
"WF3 4AH",
"LN4 4XP",
"CW5 5XG",
"LD3 9LY",
"BA12 6BR",
"SE8 3DA",
"IP14 5SS"]

postcodes = [postcode.lower().replace(' ','') for postcode in postcodes]

warnings.simplefilter('ignore')

auto_trader_df['href'] = auto_trader_data['href'].apply(lambda url: url[4:])
auto_trader_df['href'] = auto_trader_df['href'].apply(lambda url: f"https://{url}")

NameError: name 'postcodes' is not defined

In [6]:
auto_trader_df

,Unnamed: 0,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,ID,ID_final,make,model
0,0,2295,2006,Hatchback,49000,1.6,111BHP,Manual,Petrol,3 owners,Garden City,https://autotrader.co.uk/car-details/202212232...,202212122463037,"£2,29520221212246303749,000 miles",Renault,Megane
1,1,2995,2011,Hatchback,83000,2.0,160PS,Automatic,Diesel,2 owners,Liverpool,https://autotrader.co.uk/car-details/202212222...,202212122463037,"£2,99520221212246303783,000 miles",Vauxhall,Astra
2,2,3695,2008,Saloon,74000,2.0,143PS,Manual,Petrol,4 owners,Garden City,https://autotrader.co.uk/car-details/202212282...,202212122463037,"£3,69520221212246303774,000 miles",BMW,3 Series
3,3,2995,2010,Hatchback,93000,2.0,130PS,Manual,Diesel,6 owners,Garden City,https://autotrader.co.uk/car-details/202212272...,202212122463037,"£2,99520221212246303793,000 miles",Vauxhall,Insignia
4,4,2915,2012,Hatchback,85000,1.2,75PS,Manual,Petrol,2 owners,Runcorn,https://autotrader.co.uk/car-details/202212132...,202212122463037,"£2,91520221212246303785,000 miles",Renault,Clio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,12279,2018,MPV,37000,1.3,140PS,Manual,Petrol,1 owner,Derby,https://autotrader.co.uk/car-details/202211151...,202212122463037,"£12,27920221212246303737,000 miles",Renault,Scenic
7996,7996,8679,2016,SUV,75000,1.6,99PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202210271...,202212122463037,"£8,67920221212246303775,000 miles",Peugeot,2008
7997,7997,10979,2018,SUV,87400,1.5,110PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202212032...,202212122463037,"£10,97920221212246303787,400 miles",Nissan,Qashqai
7998,7998,2195,2012,Hatchback,124195,1.4,88PS,Manual,Petrol,3 owners,Burton-On-Trent,https://autotrader.co.uk/car-details/202211221...,202212122463037,"£2,195202212122463037124,195 miles",Nissan,Note


In [7]:
driver_path = './geckodriver'
firefox_options = Options()
firefox_options.headless = True
driver = webdriver.Firefox(executable_path=driver_path, options=firefox_options)
driver.get("https://autotrader.co.uk")

WebDriverWait(driver,10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id='sp_message_iframe_687971']")))
WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[aria-label='Accept All']"))).click()

time.sleep(4)

combined_mpg = []
for index, listing_url in enumerate(auto_trader_df['href']):
    print(f"Completed...: {round(((index/len(auto_trader_df))*100),2)}%")
    try:
        driver.get(listing_url)
        
        driver.execute_script("window.scrollTo(100,800);")
        
        span_tag = WebDriverWait(driver,4).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Running costs')]")))
        span_tag.find_element(By.XPATH, "..").click()

        mpg = WebDriverWait(driver,4).until(EC.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), 'mpg')]")))[-1]
        
        print(mpg.text)
        
        combined_mpg.append(mpg.text)
        
    except:
        combined_mpg.append('null')
        
print("Completed!")

auto_trader_df['combined_mpg'] = combined_mpg

Completed...: 0.0%
40.9 mpg
Completed...: 0.01%
48.70 mpg
Completed...: 0.03%
47.90 mpg
Completed...: 0.04%
57.60 mpg
Completed...: 0.05%
48.70 mpg
Completed...: 0.06%
54.30 mpg
Completed...: 0.07%
48.70 mpg
Completed...: 0.09%
62.80 mpg
Completed...: 0.1%
52.30 mpg
Completed...: 0.11%
43.50 mpg
Completed...: 0.12%
55.40 mpg
Completed...: 0.14%
58.90 mpg
Completed...: 0.15%
34.9 mpg
Completed...: 0.16%
55.40 mpg
Completed...: 0.18%
49.6 mpg
Completed...: 0.19%
42.80 mpg
Completed...: 0.2%
60.10 mpg
Completed...: 0.21%
78.50 mpg
Completed...: 0.22%
57.60 mpg
Completed...: 0.24%
47.1 mpg
Completed...: 0.25%
49.6 mpg
Completed...: 0.26%
47.10 mpg
Completed...: 0.27%
42.8 mpg
Completed...: 0.29%
64.20 mpg
Completed...: 0.3%
67.30 mpg
Completed...: 0.31%
34.9 mpg
Completed...: 0.33%
67.30 mpg
Completed...: 0.34%
60.1 mpg
Completed...: 0.35%
47.10 mpg
Completed...: 0.36%
Completed...: 0.38%
40.4 mpg
Completed...: 0.39%
41.50 mpg
Completed...: 0.4%
55.40 mpg
Completed...: 0.41%
55.40 mpg
Comp

56.50 mpg
Completed...: 3.49%
68.90 mpg
Completed...: 3.5%
40.40 mpg
Completed...: 3.51%
62.80 mpg
Completed...: 3.52%
48.70 mpg
Completed...: 3.54%
48.70 mpg
Completed...: 3.55%
48.70 mpg
Completed...: 3.56%
88.30 mpg
Completed...: 3.57%
Completed...: 3.59%
57.60 mpg
Completed...: 3.6%
70.60 mpg
Completed...: 3.61%
49.56 mpg
Completed...: 3.62%
44.80 mpg
Completed...: 3.64%
28.5 mpg
Completed...: 3.65%
25.0 mpg
Completed...: 3.66%
78.50 mpg
Completed...: 3.67%
36.70 mpg
Completed...: 3.69%
80.70 mpg
Completed...: 3.7%
31.40 mpg
Completed...: 3.71%
38.20 mpg
Completed...: 3.72%
41.50 mpg
Completed...: 3.74%
53.30 mpg
Completed...: 3.75%
56.50 mpg
Completed...: 3.76%
57.60 mpg
Completed...: 3.77%
65.70 mpg
Completed...: 3.79%
46.30 mpg
Completed...: 3.8%
47.10 mpg
Completed...: 3.81%
58.90 mpg
Completed...: 3.82%
68.90 mpg
Completed...: 3.84%
62.80 mpg
Completed...: 3.85%
34.9 mpg
Completed...: 3.86%
49.60 mpg
Completed...: 3.88%
67.30 mpg
Completed...: 3.89%
51.50 mpg
Completed...: 3.9

39.80 mpg
Completed...: 6.98%
62.80 mpg
Completed...: 6.99%
41.50 mpg
Completed...: 7.0%
83.10 mpg
Completed...: 7.01%
74.30 mpg
Completed...: 7.03%
57.60 mpg
Completed...: 7.04%
58.90 mpg
Completed...: 7.05%
53.30 mpg
Completed...: 7.06%
43.50 mpg
Completed...: 7.07%
48.70 mpg
Completed...: 7.09%
51.30 mpg
Completed...: 7.1%
45.60 mpg
Completed...: 7.11%
70.60 mpg
Completed...: 7.12%
60.10 mpg
Completed...: 7.14%
62.80 mpg
Completed...: 7.15%
Completed...: 7.16%
46.00 mpg
Completed...: 7.17%
39.80 mpg
Completed...: 7.19%
45.60 mpg
Completed...: 7.2%
60.10 mpg
Completed...: 7.21%
65.70 mpg
Completed...: 7.22%
56.50 mpg
Completed...: 7.24%
51.40 mpg
Completed...: 7.25%
50.40 mpg
Completed...: 7.26%
60.10 mpg
Completed...: 7.27%
57.60 mpg
Completed...: 7.29%
53.30 mpg
Completed...: 7.3%
47.10 mpg
Completed...: 7.31%
53.30 mpg
Completed...: 7.32%
Completed...: 7.34%
68.90 mpg
Completed...: 7.35%
60.10 mpg
Completed...: 7.36%
57.70 mpg
Completed...: 7.38%
47.10 mpg
Completed...: 7.39%
47.1

38.70 mpg
Completed...: 10.44%
61.40 mpg
Completed...: 10.45%
74.30 mpg
Completed...: 10.46%
38.20 mpg
Completed...: 10.47%

Completed...: 10.49%
64.20 mpg
Completed...: 10.5%
58.90 mpg
Completed...: 10.51%
45.50 mpg
Completed...: 10.53%
54.30 mpg
Completed...: 10.54%
46.30 mpg
Completed...: 10.55%
34.40 mpg
Completed...: 10.56%
45.60 mpg
Completed...: 10.57%
38.20 mpg
Completed...: 10.59%
57.60 mpg
Completed...: 10.6%
45.60 mpg
Completed...: 10.61%
51.40 mpg
Completed...: 10.62%
47.10 mpg
Completed...: 10.64%
58.90 mpg
Completed...: 10.65%
48.70 mpg
Completed...: 10.66%
42.70 mpg
Completed...: 10.67%
58.90 mpg
Completed...: 10.69%
45.60 mpg
Completed...: 10.7%
42.8 mpg
Completed...: 10.71%
47.90 mpg
Completed...: 10.72%
58.85 mpg
Completed...: 10.74%
40.90 mpg
Completed...: 10.75%
62.80 mpg
Completed...: 10.76%
32.80 mpg
Completed...: 10.78%
36.70 mpg
Completed...: 10.79%
29.10 mpg
Completed...: 10.8%
47.90 mpg
Completed...: 10.81%
282.50 mpg
Completed...: 10.82%
74.30 mpg
Completed..

40.90 mpg
Completed...: 13.76%
29.1 mpg
Completed...: 13.78%
49.6 mpg
Completed...: 13.79%
65.70 mpg
Completed...: 13.8%
58.90 mpg
Completed...: 13.81%
Completed...: 13.83%
35.8 mpg
Completed...: 13.84%
55.4 mpg
Completed...: 13.85%
40.4 mpg
Completed...: 13.86%
50.40 mpg
Completed...: 13.88%
55.40 mpg
Completed...: 13.89%
42.2 mpg
Completed...: 13.9%
58.90 mpg
Completed...: 13.91%
49.6 mpg
Completed...: 13.93%
37.20 mpg
Completed...: 13.94%
65.70 mpg
Completed...: 13.95%
55.40 mpg
Completed...: 13.96%
30.70 mpg
Completed...: 13.98%
55.40 mpg
Completed...: 13.99%
42.00 mpg
Completed...: 14.0%
Completed...: 14.01%
64.2 mpg
Completed...: 14.03%
44.10 mpg
Completed...: 14.04%
48.70 mpg
Completed...: 14.05%
70.60 mpg
Completed...: 14.06%
47.10 mpg
Completed...: 14.07%
Completed...: 14.09%
62.80 mpg
Completed...: 14.1%
60.10 mpg
Completed...: 14.11%
65.70 mpg
Completed...: 14.12%
52.3 mpg
Completed...: 14.14%
47.90 mpg
Completed...: 14.15%
Completed...: 14.16%
62.80 mpg
Completed...: 14.17%

...

34.90 mpg
Completed...: 85.45%
46.31 mpg
Completed...: 85.46%
44.10 mpg
Completed...: 85.47%
34.90 mpg
Completed...: 85.49%
65.70 mpg
Completed...: 85.5%
Completed...: 85.51%
65.70 mpg
Completed...: 85.52%
39.20 mpg
Completed...: 85.54%
Completed...: 85.55%
38.20 mpg
Completed...: 85.56%
65.70 mpg
Completed...: 85.58%
61.40 mpg
Completed...: 85.59%
37.20 mpg
Completed...: 85.6%
70.6 mpg
Completed...: 85.61%
43.50 mpg
Completed...: 85.62%
48.70 mpg
Completed...: 85.64%
37.70 mpg
Completed...: 85.65%
47.90 mpg
Completed...: 85.66%
32.10 mpg
Completed...: 85.67%
Completed...: 85.69%
56.50 mpg
Completed...: 85.7%
44.80 mpg
Completed...: 85.71%
57.70 mpg
Completed...: 85.72%
64.20 mpg
Completed...: 85.74%
56.50 mpg
Completed...: 85.75%
56.50 mpg
Completed...: 85.76%
74.30 mpg
Completed...: 85.78%
44.10 mpg
Completed...: 85.79%
61.40 mpg
Completed...: 85.8%
67.30 mpg
Completed...: 85.81%
48.70 mpg
Completed...: 85.82%
44.80 mpg
Completed...: 85.84%
36.20 mpg
Completed...: 85.85%
48.70 mpg
Co

35.30 mpg
Completed...: 88.9%
70.60 mpg
Completed...: 88.91%
30.1 mpg
Completed...: 88.92%
36.20 mpg
Completed...: 88.94%
45.60 mpg
Completed...: 88.95%
47.10 mpg
Completed...: 88.96%
32.50 mpg
Completed...: 88.98%
44.10 mpg
Completed...: 88.99%
60.10 mpg
Completed...: 89.0%
54.30 mpg
Completed...: 89.01%
57.60 mpg
Completed...: 89.03%
42.20 mpg
Completed...: 89.04%
50.40 mpg
Completed...: 89.05%
34.90 mpg
Completed...: 89.06%
54.30 mpg
Completed...: 89.08%
55.40 mpg
Completed...: 89.09%
50.40 mpg
Completed...: 89.1%
41.50 mpg
Completed...: 89.11%
38.20 mpg
Completed...: 89.12%
Completed...: 89.14%
53.30 mpg
Completed...: 89.15%
44.30 mpg
Completed...: 89.16%
70.60 mpg
Completed...: 89.18%
54.30 mpg
Completed...: 89.19%
57.60 mpg
Completed...: 89.2%
57.60 mpg
Completed...: 89.21%
31.50 mpg
Completed...: 89.22%
46.30 mpg
Completed...: 89.24%
Completed...: 89.25%
42.20 mpg
Completed...: 89.26%
47.90 mpg
Completed...: 89.28%
40.40 mpg
Completed...: 89.29%
56.50 mpg
Completed...: 89.3%
52.

47.90 mpg
Completed...: 92.38%
35.80 mpg
Completed...: 92.39%
61.40 mpg
Completed...: 92.4%
Completed...: 92.41%
64.20 mpg
Completed...: 92.42%
Completed...: 92.44%
62.80 mpg
Completed...: 92.45%
Completed...: 92.46%
44.10 mpg
Completed...: 92.47%
67.30 mpg
Completed...: 92.49%
47.9 mpg
Completed...: 92.5%
47.90 mpg
Completed...: 92.51%
53.30 mpg
Completed...: 92.53%
64.20 mpg
Completed...: 92.54%
41.50 mpg
Completed...: 92.55%
39.20 mpg
Completed...: 92.56%
56.50 mpg
Completed...: 92.58%
44.80 mpg
Completed...: 92.59%
64.20 mpg
Completed...: 92.6%
54.30 mpg
Completed...: 92.61%
51.40 mpg
Completed...: 92.62%
67.30 mpg
Completed...: 92.64%
Completed...: 92.65%
53.30 mpg
Completed...: 92.66%
68.90 mpg
Completed...: 92.67%
68.90 mpg
Completed...: 92.69%
47.90 mpg
Completed...: 92.7%
62.80 mpg
Completed...: 92.71%
44.10 mpg
Completed...: 92.73%
58.90 mpg
Completed...: 92.74%
34.0 mpg
Completed...: 92.75%
57.60 mpg
Completed...: 92.76%
46.30 mpg
Completed...: 92.77%
57.60 mpg
Completed...:

47.90 mpg
Completed...: 95.84%
65.69 mpg
Completed...: 95.85%
44.10 mpg
Completed...: 95.86%
62.80 mpg
Completed...: 95.88%
Completed...: 95.89%
34.4 mpg
Completed...: 95.9%
54.30 mpg
Completed...: 95.91%
54.30 mpg
Completed...: 95.93%
55.40 mpg
Completed...: 95.94%
80.70 mpg
Completed...: 95.95%
134.50 mpg
Completed...: 95.96%
62.80 mpg
Completed...: 95.97%
62.80 mpg
Completed...: 95.99%
62.80 mpg
Completed...: 96.0%
37.20 mpg
Completed...: 96.01%
54.30 mpg
Completed...: 96.03%
44.8 mpg
Completed...: 96.04%
47.90 mpg
Completed...: 96.05%
58.90 mpg
Completed...: 96.06%
37.70 mpg
Completed...: 96.08%
32.50 mpg
Completed...: 96.09%
Completed...: 96.1%
41.50 mpg
Completed...: 96.11%
43.93 mpg
Completed...: 96.12%
53.30 mpg
Completed...: 96.14%
62.8 mpg
Completed...: 96.15%
57.70 mpg
Completed...: 96.16%
62.80 mpg
Completed...: 96.17%
53.30 mpg
Completed...: 96.19%
31.7 mpg
Completed...: 96.2%
201.80 mpg
Completed...: 96.21%
65.70 mpg
Completed...: 96.23%
42.20 mpg
Completed...: 96.24%
35.

44.1 mpg
Completed...: 99.31%
31.70 mpg
Completed...: 99.33%
44.10 mpg
Completed...: 99.34%
70.60 mpg
Completed...: 99.35%
38.70 mpg
Completed...: 99.36%
65.70 mpg
Completed...: 99.38%
50.40 mpg
Completed...: 99.39%
41.50 mpg
Completed...: 99.4%
53.30 mpg
Completed...: 99.41%
54.30 mpg
Completed...: 99.42%
55.40 mpg
Completed...: 99.44%
34.9 mpg
Completed...: 99.45%
78.50 mpg
Completed...: 99.46%
68.90 mpg
Completed...: 99.48%
54.30 mpg
Completed...: 99.49%
64.20 mpg
Completed...: 99.5%
58.90 mpg
Completed...: 99.51%
57.70 mpg
Completed...: 99.52%
Completed...: 99.54%
65.69 mpg
Completed...: 99.55%
Completed...: 99.56%
61.40 mpg
Completed...: 99.58%
37.20 mpg
Completed...: 99.59%
64.20 mpg
Completed...: 99.6%
55.60 mpg
Completed...: 99.61%
47.10 mpg
Completed...: 99.62%
47.10 mpg
Completed...: 99.64%
40.40 mpg
Completed...: 99.65%
56.50 mpg
Completed...: 99.66%
39.20 mpg
Completed...: 99.67%
48.70 mpg
Completed...: 99.69%
36.70 mpg
Completed...: 99.7%
44.80 mpg
Completed...: 99.71%
61.

In [61]:
auto_trader_df['price'] = auto_trader_df.price.str.replace('£','')
auto_trader_df['price'] = auto_trader_df.price.str.replace(',','')
auto_trader_df['price'] = pd.to_numeric(auto_trader_df['price'])

auto_trader_df['year'] = auto_trader_df['year'].apply(lambda year: year[:4])
auto_trader_df['year'] = pd.to_numeric(auto_trader_df['year'])

auto_trader_df['mileage'] = auto_trader_df['mileage'].apply(lambda mileage: mileage[:-6])
auto_trader_df['mileage'] = auto_trader_df.mileage.str.replace(',','')
auto_trader_df['mileage'] = pd.to_numeric(auto_trader_df['mileage'])

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

# Engine size is shown in Litres
auto_trader_df['engine_size'] = auto_trader_df.engine_size.str.replace('L','')
auto_trader_df['engine_size'] = pd.to_numeric(auto_trader_df['engine_size'])

auto_trader_df['number_of_owners'] = auto_trader_df['number_of_owners'].apply(lambda num_of_owners: num_of_owners[0])
auto_trader_df['number_of_owners'] = pd.to_numeric(auto_trader_df['number_of_owners'], errors='coerce')

auto_trader_df['location'] = auto_trader_df['location'].str.replace('[0-9].[0-9]', 'null', regex=True).str.strip()

# Power will be converted to BHP if the value is in PS units
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: int(power[:-2])/1.014 if 'PS' in power else power)
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: power[:-3] if 'BHP' in str(power) else power)
auto_trader_df['power'] = pd.to_numeric(auto_trader_df['power'], errors='coerce')

auto_trader_df['combined_mpg'] = auto_trader_df['combined_mpg'].apply(lambda mpg: mpg.split()[0] if mpg != 'null' else mpg)
auto_trader_df['make'] = auto_trader_df['make_model'].apply(lambda make: make.split()[0])
auto_trader_df['model'] = auto_trader_df['make_model'].apply(lambda make: ' '.join(make.split()[1:]))
auto_trader_df.drop(columns='make_model',inplace=True)

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

for index,make in enumerate(auto_trader_df['make']):
    if re.match(r"Mazda[0-9]+",make):
        auto_trader_df['make'][index] = re.split('(\d+)',make)[0]
        auto_trader_df['model'][index] = re.split('(\d+)',make)[1]
    else:
        pass

AttributeError: Can only use .str accessor with string values!

In [25]:
auto_trader_df.to_csv("auto_trader_data_8000_rows")

In [248]:
auto_trader_df = pd.read_csv(r"/Users/patrykpacholek/Desktop/auto_trader_data_8000_rows")

In [249]:
auto_trader_df

,Unnamed: 0.1,Unnamed: 0,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,ID,ID_final,make,model,combined_mpg
0,0,0,2295,2006,Hatchback,49000,1.6,111BHP,Manual,Petrol,3 owners,Garden City,https://autotrader.co.uk/car-details/202212232...,202212122463037,"£2,29520221212246303749,000 miles",Renault,Megane,40.9 mpg
1,1,1,2995,2011,Hatchback,83000,2.0,160PS,Automatic,Diesel,2 owners,Liverpool,https://autotrader.co.uk/car-details/202212222...,202212122463037,"£2,99520221212246303783,000 miles",Vauxhall,Astra,48.70 mpg
2,2,2,3695,2008,Saloon,74000,2.0,143PS,Manual,Petrol,4 owners,Garden City,https://autotrader.co.uk/car-details/202212282...,202212122463037,"£3,69520221212246303774,000 miles",BMW,3 Series,47.90 mpg
3,3,3,2995,2010,Hatchback,93000,2.0,130PS,Manual,Diesel,6 owners,Garden City,https://autotrader.co.uk/car-details/202212272...,202212122463037,"£2,99520221212246303793,000 miles",Vauxhall,Insignia,57.60 mpg
4,4,4,2915,2012,Hatchback,85000,1.2,75PS,Manual,Petrol,2 owners,Runcorn,https://autotrader.co.uk/car-details/202212132...,202212122463037,"£2,91520221212246303785,000 miles",Renault,Clio,48.70 mpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,7995,12279,2018,MPV,37000,1.3,140PS,Manual,Petrol,1 owner,Derby,https://autotrader.co.uk/car-details/202211151...,202212122463037,"£12,27920221212246303737,000 miles",Renault,Scenic,41.50 mpg
7996,7996,7996,8679,2016,SUV,75000,1.6,99PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202210271...,202212122463037,"£8,67920221212246303775,000 miles",Peugeot,2008,76.30 mpg
7997,7997,7997,10979,2018,SUV,87400,1.5,110PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202212032...,202212122463037,"£10,97920221212246303787,400 miles",Nissan,Qashqai,74.30 mpg
7998,7998,7998,2195,2012,Hatchback,124195,1.4,88PS,Manual,Petrol,3 owners,Burton-On-Trent,https://autotrader.co.uk/car-details/202211221...,202212122463037,"£2,195202212122463037124,195 miles",Nissan,Note,47.90 mpg


In [250]:
unwanted_elements = [element for element in auto_trader_df.engine_size.unique().tolist() if '.' not in element]
for index in range(len(auto_trader_df)):
    specs_list = []
    if auto_trader_df['engine_size'][index] in unwanted_elements:
        
        specs_list.append(auto_trader_df['engine_size'][index])
        specs_list.append(auto_trader_df['power'][index])
        specs_list.append(auto_trader_df['gearbox'][index])
        specs_list.append(auto_trader_df['fuel_type'][index])

        
        auto_trader_df['power'][index] = specs_list[0]
        auto_trader_df['gearbox'][index] = specs_list[1]
        auto_trader_df['fuel_type'][index] = specs_list[2]
        auto_trader_df['number_of_owners'][index] = specs_list[3]
        auto_trader_df['engine_size'][index] = 'null'

In [260]:
unwanted_elements = [element for element in power_list if 'HP' not in element and 'PS' not in element]
for index in range(len(auto_trader_df)):
    specs_list = []
    if auto_trader_df['power'][index] in unwanted_elements:
        
        specs_list.append(auto_trader_df['power'][index])
        specs_list.append(auto_trader_df['gearbox'][index])
        specs_list.append(auto_trader_df['fuel_type'][index])
        
        auto_trader_df['gearbox'][index] = specs_list[0]
        auto_trader_df['fuel_type'][index] = specs_list[1]
        auto_trader_df['number_of_owners'][index] = specs_list[2]
        auto_trader_df['power'][index] = 'null'

In [277]:
unwanted_elements = [element for element in auto_trader_df.number_of_owners.unique().tolist() if 'owner' not in element]
for index in range(len(auto_trader_df)):
    if auto_trader_df['number_of_owners'][index] in unwanted_elements:
        auto_trader_df['number_of_owners'][index] = 'null'

In [281]:
auto_trader_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'price', 'year', 'type', 'mileage',
       'engine_size', 'power', 'gearbox', 'fuel_type', 'number_of_owners',
       'location', 'href', 'ID', 'ID_final', 'make', 'model', 'combined_mpg'],
      dtype='object')

In [282]:
auto_trader_df.drop(columns=['Unnamed: 0.1','Unnamed: 0','ID','ID_final'],inplace=True)

In [283]:
auto_trader_df

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,combined_mpg
0,2295,2006,Hatchback,49000,1.6,111BHP,Manual,Petrol,3 owners,Garden City,https://autotrader.co.uk/car-details/202212232...,Renault,Megane,40.9 mpg
1,2995,2011,Hatchback,83000,2.0,160PS,Automatic,Diesel,2 owners,Liverpool,https://autotrader.co.uk/car-details/202212222...,Vauxhall,Astra,48.70 mpg
2,3695,2008,Saloon,74000,2.0,143PS,Manual,Petrol,4 owners,Garden City,https://autotrader.co.uk/car-details/202212282...,BMW,3 Series,47.90 mpg
3,2995,2010,Hatchback,93000,2.0,130PS,Manual,Diesel,6 owners,Garden City,https://autotrader.co.uk/car-details/202212272...,Vauxhall,Insignia,57.60 mpg
4,2915,2012,Hatchback,85000,1.2,75PS,Manual,Petrol,2 owners,Runcorn,https://autotrader.co.uk/car-details/202212132...,Renault,Clio,48.70 mpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,12279,2018,MPV,37000,1.3,140PS,Manual,Petrol,1 owner,Derby,https://autotrader.co.uk/car-details/202211151...,Renault,Scenic,41.50 mpg
7996,8679,2016,SUV,75000,1.6,99PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202210271...,Peugeot,2008,76.30 mpg
7997,10979,2018,SUV,87400,1.5,110PS,Manual,Diesel,2 owners,Derby,https://autotrader.co.uk/car-details/202212032...,Nissan,Qashqai,74.30 mpg
7998,2195,2012,Hatchback,124195,1.4,88PS,Manual,Petrol,3 owners,Burton-On-Trent,https://autotrader.co.uk/car-details/202211221...,Nissan,Note,47.90 mpg


In [285]:
# Power will be converted to BHP if the value is in PS units
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: int(power[:-2])/1.014 if 'PS' in power else power)
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: power[:-3] if 'BHP' in str(power) else power)
auto_trader_df['power'] = pd.to_numeric(auto_trader_df['power'], errors='coerce')

In [287]:
auto_trader_df['number_of_owners'] = auto_trader_df['number_of_owners'].apply(lambda num_of_owners: num_of_owners[0])
auto_trader_df['number_of_owners'] = pd.to_numeric(auto_trader_df['number_of_owners'], errors='coerce')

In [291]:
auto_trader_df['engine_size'] = pd.to_numeric(auto_trader_df['engine_size'],errors='coerce')

In [364]:
for index,make in enumerate(auto_trader_df['make']):
    if re.match(r"MAZDA[0-9]+",make):
        auto_trader_df['make'][index] = re.split('(\d+)',make)[0]
        auto_trader_df['model'][index] = re.split('(\d+)',make)[1]
    else:
        pass

In [386]:
auto_trader_df['cmpg'] = auto_trader_df['combined_mpg'].str.replace(r' mpg', '')

In [389]:
auto_trader_df['cmpg'] = pd.to_numeric(auto_trader_df['cmpg'])

In [392]:
auto_trader_df.drop(columns='combined_mpg', inplace=True)

In [393]:
auto_trader_df

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,cmpg
0,2295,2006,Hatchback,49000,1.6,111.000000,Manual,Petrol,3.0,Garden City,https://autotrader.co.uk/car-details/202212232...,Renault,Megane,40.9
1,2995,2011,Hatchback,83000,2.0,157.790927,Automatic,Diesel,2.0,Liverpool,https://autotrader.co.uk/car-details/202212222...,Vauxhall,Astra,48.7
2,3695,2008,Saloon,74000,2.0,141.025641,Manual,Petrol,4.0,Garden City,https://autotrader.co.uk/car-details/202212282...,BMW,3 Series,47.9
3,2995,2010,Hatchback,93000,2.0,128.205128,Manual,Diesel,6.0,Garden City,https://autotrader.co.uk/car-details/202212272...,Vauxhall,Insignia,57.6
4,2915,2012,Hatchback,85000,1.2,73.964497,Manual,Petrol,2.0,Runcorn,https://autotrader.co.uk/car-details/202212132...,Renault,Clio,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,12279,2018,MPV,37000,1.3,138.067061,Manual,Petrol,1.0,Derby,https://autotrader.co.uk/car-details/202211151...,Renault,Scenic,41.5
7996,8679,2016,SUV,75000,1.6,97.633136,Manual,Diesel,2.0,Derby,https://autotrader.co.uk/car-details/202210271...,Peugeot,2008,76.3
7997,10979,2018,SUV,87400,1.5,108.481262,Manual,Diesel,2.0,Derby,https://autotrader.co.uk/car-details/202212032...,Nissan,Qashqai,74.3
7998,2195,2012,Hatchback,124195,1.4,86.785010,Manual,Petrol,3.0,Burton-On-Trent,https://autotrader.co.uk/car-details/202211221...,Nissan,Note,47.9


In [394]:
auto_trader_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             8000 non-null   int64  
 1   year              8000 non-null   int64  
 2   type              8000 non-null   object 
 3   mileage           8000 non-null   int64  
 4   engine_size       7993 non-null   float64
 5   power             7987 non-null   float64
 6   gearbox           8000 non-null   object 
 7   fuel_type         8000 non-null   object 
 8   number_of_owners  7288 non-null   float64
 9   location          7962 non-null   object 
 10  href              8000 non-null   object 
 11  make              8000 non-null   object 
 12  model             7999 non-null   object 
 13  cmpg              7466 non-null   float64
dtypes: float64(4), int64(3), object(7)
memory usage: 875.1+ KB


In [399]:
auto_trader_df.to_csv("processed_auto_trader_data_8000_rows")

In [304]:
processed_auto_trader_df = pd.read_csv(r"/Users/patrykpacholek/Desktop/processed_auto_trader_data_8000_rows")

In [316]:
processed_auto_trader_df['combined_mpg'].isna().sum()

534

In [321]:
auto_trader_df_10100 = pd.read_excel(r"/Users/patrykpacholek/Desktop/processed_auto_trader_data_10000_rows.xlsx")

In [322]:
auto_trader_df_10100

,Unnamed: 0.1,Unnamed: 0,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,ID,ID_final,make,model,combined_mpg
0,8000,8000,6648,2011,Estate,80589,1.6,112BHP,Manual,Diesel,6 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022122126898...,202212122463037,"£6,64820221212246303780,589 miles",MINI,Clubman,72.40 mpg
1,8001,8001,12279,2018,SUV,43000,1.6,132PS,Manual,Petrol,1 owner,Derby,www.autotrader.co.uk/car-details/2022111517059...,202212122463037,"£12,27920221212246303743,000 miles",Hyundai,Tucson,44.80 mpg
2,8002,8002,9948,2013,SUV,54993,2.0,138BHP,Manual,Diesel,3 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022100403955...,202212122463037,"£9,94820221212246303754,993 miles",Ford,Kuga,53.30 mpg
3,8003,8003,9448,2014,Saloon,52743,2.1,168BHP,Automatic,Diesel,3 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022091296776...,202212122463037,"£9,44820221212246303752,743 miles",Infiniti,Q50,58.90 mpg
4,8004,8004,5979,2014,Hatchback,28600,1.2,87PS,Manual,Petrol,2 owners,Derby,www.autotrader.co.uk/car-details/2022100705206...,202212122463037,"£5,97920221212246303728,600 miles",Hyundai,i10,57.60 mpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,18095,18095,37794,2020,Pickup,21243,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2022121324827...,202212122463037,"£37,79420221212246303721,243 miles",Mitsubishi,L200,29.10 mpg
10096,18096,18096,36594,2020,Pickup,25000,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2022120121730...,202212122463037,"£36,59420221212246303725,000 miles",Mitsubishi,L200,29.10 mpg
10097,18097,18097,37794,2020,Pickup,18966,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2021100782163...,202212122463037,"£37,79420221212246303718,966 miles",Mitsubishi,L200,29.10 mpg
10098,18098,18098,37794,2020,Pickup,10503,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2021081161027...,202212122463037,"£37,79420221212246303710,503 miles",Mitsubishi,L200,29.10 mpg


In [333]:
unwanted_elements = [element for element in auto_trader_df_10100.engine_size.unique().tolist() if '.' not in element]
for index in range(len(auto_trader_df_10100)):
    specs_list = []
    if auto_trader_df_10100['engine_size'][index] in unwanted_elements:
        
        specs_list.append(auto_trader_df_10100['engine_size'][index])
        specs_list.append(auto_trader_df_10100['power'][index])
        specs_list.append(auto_trader_df_10100['gearbox'][index])
        specs_list.append(auto_trader_df_10100['fuel_type'][index])

        
        auto_trader_df_10100['power'][index] = specs_list[0]
        auto_trader_df_10100['gearbox'][index] = specs_list[1]
        auto_trader_df_10100['fuel_type'][index] = specs_list[2]
        auto_trader_df_10100['number_of_owners'][index] = specs_list[3]
        auto_trader_df_10100['engine_size'][index] = 'null'

In [339]:
unwanted_elements = [element for element in auto_trader_df_10100.power.unique().tolist() if 'HP' not in element and 'PS' not in element]
for index in range(len(auto_trader_df_10100)):
    specs_list = []
    if auto_trader_df_10100['power'][index] in unwanted_elements:
        
        specs_list.append(auto_trader_df_10100['power'][index])
        specs_list.append(auto_trader_df_10100['gearbox'][index])
        specs_list.append(auto_trader_df_10100['fuel_type'][index])
        
        auto_trader_df_10100['gearbox'][index] = specs_list[0]
        auto_trader_df_10100['fuel_type'][index] = specs_list[1]
        auto_trader_df_10100['number_of_owners'][index] = specs_list[2]
        auto_trader_df_10100['power'][index] = 'null'

In [342]:
unwanted_elements = [element for element in auto_trader_df_10100.number_of_owners.unique().tolist() if 'owner' not in element]
for index in range(len(auto_trader_df_10100)):
    if auto_trader_df_10100['number_of_owners'][index] in unwanted_elements:
        auto_trader_df_10100['number_of_owners'][index] = 'null'

In [344]:
auto_trader_df_10100.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'price', 'year', 'type', 'mileage',
       'engine_size', 'power', 'gearbox', 'fuel_type', 'number_of_owners',
       'location', 'href', 'ID', 'ID_final', 'make', 'model', 'combined_mpg'],
      dtype='object')

In [345]:
auto_trader_df_10100.drop(columns=['ID','ID_final','Unnamed: 0.1','Unnamed: 0'],inplace=True)

In [346]:
auto_trader_df_10100

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,combined_mpg
0,6648,2011,Estate,80589,1.6,112BHP,Manual,Diesel,6 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022122126898...,MINI,Clubman,72.40 mpg
1,12279,2018,SUV,43000,1.6,132PS,Manual,Petrol,1 owner,Derby,www.autotrader.co.uk/car-details/2022111517059...,Hyundai,Tucson,44.80 mpg
2,9948,2013,SUV,54993,2.0,138BHP,Manual,Diesel,3 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022100403955...,Ford,Kuga,53.30 mpg
3,9448,2014,Saloon,52743,2.1,168BHP,Automatic,Diesel,3 owners,Burton On Trent,www.autotrader.co.uk/car-details/2022091296776...,Infiniti,Q50,58.90 mpg
4,5979,2014,Hatchback,28600,1.2,87PS,Manual,Petrol,2 owners,Derby,www.autotrader.co.uk/car-details/2022100705206...,Hyundai,i10,57.60 mpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,37794,2020,Pickup,21243,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2022121324827...,Mitsubishi,L200,29.10 mpg
10096,36594,2020,Pickup,25000,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2022120121730...,Mitsubishi,L200,29.10 mpg
10097,37794,2020,Pickup,18966,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2021100782163...,Mitsubishi,L200,29.10 mpg
10098,37794,2020,Pickup,10503,2.2,150PS,Automatic,Diesel,1 owner,Dumfries,www.autotrader.co.uk/car-details/2021081161027...,Mitsubishi,L200,29.10 mpg


In [347]:
# Power will be converted to BHP if the value is in PS units
auto_trader_df_10100['power'] = auto_trader_df_10100['power'].apply(lambda power: int(power[:-2])/1.014 if 'PS' in power else power)
auto_trader_df_10100['power'] = auto_trader_df_10100['power'].apply(lambda power: power[:-3] if 'BHP' in str(power) else power)
auto_trader_df_10100['power'] = pd.to_numeric(auto_trader_df_10100['power'], errors='coerce')

In [348]:
auto_trader_df_10100['number_of_owners'] = auto_trader_df_10100['number_of_owners'].apply(lambda num_of_owners: num_of_owners[0])
auto_trader_df_10100['number_of_owners'] = pd.to_numeric(auto_trader_df_10100['number_of_owners'], errors='coerce')

In [349]:
auto_trader_df_10100['engine_size'] = pd.to_numeric(auto_trader_df_10100['engine_size'],errors='coerce')

In [355]:
for index,make in enumerate(auto_trader_df_10100['make']):
    if re.match(r"MAZDA[0-9]+",make):
        auto_trader_df_10100['make'][index] = re.split('(\d+)',make)[0]
        auto_trader_df_10100['model'][index] = re.split('(\d+)',make)[1]
    else:
        pass

In [396]:
auto_trader_df_10100['cmpg'] = auto_trader_df_10100['combined_mpg'].str.replace(r' mpg','')
auto_trader_df_10100['cmpg'] = pd.to_numeric(auto_trader_df_10100['cmpg'],errors='coerce')
auto_trader_df_10100.drop(columns='combined_mpg',inplace=True)

In [398]:
auto_trader_df_10100

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,cmpg
0,6648,2011,Estate,80589,1.6,112.000000,Manual,Diesel,6.0,Burton On Trent,www.autotrader.co.uk/car-details/2022122126898...,MINI,Clubman,72.4
1,12279,2018,SUV,43000,1.6,130.177515,Manual,Petrol,1.0,Derby,www.autotrader.co.uk/car-details/2022111517059...,Hyundai,Tucson,44.8
2,9948,2013,SUV,54993,2.0,138.000000,Manual,Diesel,3.0,Burton On Trent,www.autotrader.co.uk/car-details/2022100403955...,Ford,Kuga,53.3
3,9448,2014,Saloon,52743,2.1,168.000000,Automatic,Diesel,3.0,Burton On Trent,www.autotrader.co.uk/car-details/2022091296776...,Infiniti,Q50,58.9
4,5979,2014,Hatchback,28600,1.2,85.798817,Manual,Petrol,2.0,Derby,www.autotrader.co.uk/car-details/2022100705206...,Hyundai,i10,57.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,37794,2020,Pickup,21243,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022121324827...,Mitsubishi,L200,29.1
10096,36594,2020,Pickup,25000,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022120121730...,Mitsubishi,L200,29.1
10097,37794,2020,Pickup,18966,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021100782163...,Mitsubishi,L200,29.1
10098,37794,2020,Pickup,10503,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021081161027...,Mitsubishi,L200,29.1


In [401]:
auto_trader_final = pd.concat([auto_trader_df,auto_trader_df_10100])

In [403]:
auto_trader_final.reset_index(inplace=True)

In [405]:
auto_trader_final.drop(columns='index',inplace=True)

In [406]:
auto_trader_final

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,cmpg
0,2295,2006,Hatchback,49000,1.6,111.000000,Manual,Petrol,3.0,Garden City,https://autotrader.co.uk/car-details/202212232...,Renault,Megane,40.9
1,2995,2011,Hatchback,83000,2.0,157.790927,Automatic,Diesel,2.0,Liverpool,https://autotrader.co.uk/car-details/202212222...,Vauxhall,Astra,48.7
2,3695,2008,Saloon,74000,2.0,141.025641,Manual,Petrol,4.0,Garden City,https://autotrader.co.uk/car-details/202212282...,BMW,3 Series,47.9
3,2995,2010,Hatchback,93000,2.0,128.205128,Manual,Diesel,6.0,Garden City,https://autotrader.co.uk/car-details/202212272...,Vauxhall,Insignia,57.6
4,2915,2012,Hatchback,85000,1.2,73.964497,Manual,Petrol,2.0,Runcorn,https://autotrader.co.uk/car-details/202212132...,Renault,Clio,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18095,37794,2020,Pickup,21243,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022121324827...,Mitsubishi,L200,29.1
18096,36594,2020,Pickup,25000,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022120121730...,Mitsubishi,L200,29.1
18097,37794,2020,Pickup,18966,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021100782163...,Mitsubishi,L200,29.1
18098,37794,2020,Pickup,10503,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021081161027...,Mitsubishi,L200,29.1


In [429]:
auto_trader_final.to_csv("auto_trader_data_final")

In [453]:
auto_trader_final = pd.read_csv(r'/Users/patrykpacholek/Desktop/auto_trader_data_final')

In [456]:
auto_trader_final[auto_trader_final['make'] == 'Land Rover']

,Unnamed: 0,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,cmpg
35,35,5500,2007,SUV,75000,2.2,157.790927,Manual,Diesel,2.0,Wrexham,https://autotrader.co.uk/car-details/202208028...,Land Rover,Freelander 2,37.7
50,50,6495,2011,SUV,92000,2.2,147.928994,Manual,Diesel,3.0,Garden City,https://autotrader.co.uk/car-details/202210040...,Land Rover,Freelander 2,45.6
54,54,2000,2004,SUV,157600,3.0,174.000000,Automatic,Diesel,5.0,Runcorn,https://autotrader.co.uk/car-details/202205105...,Land Rover,Range Rover,25.0
142,142,11990,2013,SUV,74000,2.2,187.376726,Automatic,Diesel,4.0,Chester,https://autotrader.co.uk/car-details/202209229...,Land Rover,Freelander 2,40.4
215,215,8995,2012,SUV,68000,2.2,147.928994,Manual,Diesel,3.0,Garden City,https://autotrader.co.uk/car-details/202211231...,Land Rover,Freelander 2,45.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17677,17677,11995,2012,SUV,73534,2.2,187.376726,Automatic,Diesel,4.0,Glasgow,www.autotrader.co.uk/car-details/2022122026573...,Land Rover,Range Rover Evoque,NaN
17698,17698,25995,2018,SUV,36397,2.0,177.514793,Automatic,Diesel,NaN,Glasgow,www.autotrader.co.uk/car-details/2022121324925...,Land Rover,Range Rover Evoque,44.1
17864,17864,25999,2015,SUV,63000,3.0,301.775148,Automatic,Diesel,4.0,Glasgow,www.autotrader.co.uk/car-details/2022111316372...,Land Rover,Range Rover Sport,40.4
17928,17928,7975,2010,SUV,132000,3.0,241.617357,Automatic,Diesel,4.0,Glasgow,www.autotrader.co.uk/car-details/2022120222145...,Land Rover,Discovery 4,30.4


In [455]:
for index in range(len(auto_trader_final)):
    if auto_trader_final['make'][index] == 'Land':
        auto_trader_final['make'][index] = 'Land Rover'
        auto_trader_final['model'][index] = ' '.join(auto_trader_final['model'][index].split()[1:])

In [457]:
auto_trader_final.to_csv("auto_trader_data_final")

In [464]:
auto_trader_final.columns

Index(['Unnamed: 0', 'price', 'year', 'type', 'mileage', 'engine_size',
       'power', 'gearbox', 'fuel_type', 'number_of_owners', 'location', 'href',
       'make', 'model', 'cmpg'],
      dtype='object')

In [465]:
auto_trader_final.drop(columns='Unnamed: 0',inplace=True)

In [466]:
auto_trader_final

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,href,make,model,cmpg
0,2295,2006,Hatchback,49000,1.6,111.000000,Manual,Petrol,3.0,Garden City,https://autotrader.co.uk/car-details/202212232...,Renault,Megane,40.9
1,2995,2011,Hatchback,83000,2.0,157.790927,Automatic,Diesel,2.0,Liverpool,https://autotrader.co.uk/car-details/202212222...,Vauxhall,Astra,48.7
2,3695,2008,Saloon,74000,2.0,141.025641,Manual,Petrol,4.0,Garden City,https://autotrader.co.uk/car-details/202212282...,BMW,3 Series,47.9
3,2995,2010,Hatchback,93000,2.0,128.205128,Manual,Diesel,6.0,Garden City,https://autotrader.co.uk/car-details/202212272...,Vauxhall,Insignia,57.6
4,2915,2012,Hatchback,85000,1.2,73.964497,Manual,Petrol,2.0,Runcorn,https://autotrader.co.uk/car-details/202212132...,Renault,Clio,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18095,37794,2020,Pickup,21243,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022121324827...,Mitsubishi,L200,29.1
18096,36594,2020,Pickup,25000,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2022120121730...,Mitsubishi,L200,29.1
18097,37794,2020,Pickup,18966,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021100782163...,Mitsubishi,L200,29.1
18098,37794,2020,Pickup,10503,2.2,147.928994,Automatic,Diesel,1.0,Dumfries,www.autotrader.co.uk/car-details/2021081161027...,Mitsubishi,L200,29.1


In [467]:
auto_trader_final.to_csv("auto_trader_data_final")